In [ ]:
import pandas as pd

# Utils
import src.utils.visualization.gantt_diagram as gantt
from src.domain.Collection import LiveJobCollection
from src.domain.Initializer import JobsInitializer
from src.domain.Query import JobQuery
from src.utils.analysis_basics import check_constraints as check


# Data access
from configs.path_manager import get_path

pd.set_option('display.min_rows', 12)
pd.set_option('display.max_rows', 16)

In [1]:
from src.domain.Query import RoutingQuery
from src.domain.Initializer import JobsInitializer
from src.domain.Collection import LiveJobCollection

### Generation of job-shop scheduling problem and the associated jobs information

In [2]:
routings = RoutingQuery.get_by_source_name(source_name="Fisher and Thompson 10x10")

for routing in routings[:2]:
    print(routing)
    for operation in routing.operations:
        print(f" {operation}")

Routing(id='01-00', source_id=1)
 RoutingOperation(routing_id='01-00', position_number=0, machine_name='M01-00', duration=29)
 RoutingOperation(routing_id='01-00', position_number=1, machine_name='M01-01', duration=78)
 RoutingOperation(routing_id='01-00', position_number=2, machine_name='M01-02', duration=9)
 RoutingOperation(routing_id='01-00', position_number=3, machine_name='M01-03', duration=36)
 RoutingOperation(routing_id='01-00', position_number=4, machine_name='M01-04', duration=49)
 RoutingOperation(routing_id='01-00', position_number=5, machine_name='M01-05', duration=11)
 RoutingOperation(routing_id='01-00', position_number=6, machine_name='M01-06', duration=62)
 RoutingOperation(routing_id='01-00', position_number=7, machine_name='M01-07', duration=56)
 RoutingOperation(routing_id='01-00', position_number=8, machine_name='M01-08', duration=44)
 RoutingOperation(routing_id='01-00', position_number=9, machine_name='M01-09', duration=21)
Routing(id='01-01', source_id=1)
 Rout

In [3]:
jobs = JobsInitializer.create_simple_jobs(routings = routings, shuffle=False)
for job in jobs[:2]:
    print(job)
    for operation in job.operations:
        print(f" {operation}")


Job(id='01-0000', routing_id='01-00', arrival=None, earliest_start=0, deadline=None, sum_duration=395, max_bottleneck_utilization=None)
 JobOperation(job_id='01-0000', position_number=0, machine_name='M01-00', duration=29)
 JobOperation(job_id='01-0000', position_number=1, machine_name='M01-01', duration=78)
 JobOperation(job_id='01-0000', position_number=2, machine_name='M01-02', duration=9)
 JobOperation(job_id='01-0000', position_number=3, machine_name='M01-03', duration=36)
 JobOperation(job_id='01-0000', position_number=4, machine_name='M01-04', duration=49)
 JobOperation(job_id='01-0000', position_number=5, machine_name='M01-05', duration=11)
 JobOperation(job_id='01-0000', position_number=6, machine_name='M01-06', duration=62)
 JobOperation(job_id='01-0000', position_number=7, machine_name='M01-07', duration=56)
 JobOperation(job_id='01-0000', position_number=8, machine_name='M01-08', duration=44)
 JobOperation(job_id='01-0000', position_number=9, machine_name='M01-09', duration

In [4]:
jobs_collection = LiveJobCollection(jobs)
print(f"Total number of operations: {jobs_collection.count_operations()}")

Total number of operations: 100


## Scheduling

### a) Makespan

In [5]:
from src.solvers.CP_Solver import Solver

model, index_mapper, start_times, end_times = Solver.build_makespan_model(
    jobs_collection = jobs_collection,
    schedule_start= 0,
)

solver, status = Solver.solve_model(
    model=model,
    print_log_search_progress=False,
    time_limit= 120,
    gap_limit= 0,
    log_file= None
)

schedule_job_collection = Solver.get_schedule(
    index_mapper = index_mapper,
    start_times = start_times,
    end_times = end_times,
    solver = solver
)

schedule_job_collection.to_operations_dataframe()

,Job,Routing_ID,Operation,Machine,Start,Processing Time,End,Ready Time,Deadline
0,01-0000,01-00,0,M01-00,76,29,105,0,None
1,01-0000,01-00,1,M01-01,448,78,526,0,None
2,01-0000,01-00,2,M01-02,526,9,535,0,None
3,01-0000,01-00,3,M01-03,535,36,571,0,None
4,01-0000,01-00,4,M01-04,575,49,624,0,None
...,...,...,...,...,...,...,...,...,...
95,01-0009,01-09,5,M01-09,518,76,594,0,None
96,01-0009,01-09,6,M01-05,637,47,684,0,None
97,01-0009,01-09,7,M01-03,738,52,790,0,None
98,01-0009,01-09,8,M01-04,790,90,880,0,None


In [6]:
Solver.get_solver_info(
    solver = solver,
    status = status,
)

{'status': 'OPTIMAL',
 'objective_value': 930.0,
 'best_objective_bound': 930.0,
 'number_of_branches': 0,
 'wall_time': 2.86}

In [ ]:
gantt.get_plot(df_schedule, perspective="Machine")
check.check_core_schedule_constraints(df_schedule)
check.is_start_correct(df_schedule, df_jobs_arrivals)
check.is_duration_correct(df_schedule)

### b) Flow Time

In [ ]:
job_earliest_starts = get_earliest_start_dict(df_jobs_arrival_for_flowtime, earliest_start_column = "Arrival")
job_earliest_starts

In [ ]:
log_file_path = None # logs_path / "flowtime_cp.log"

schedule = solve_jssp_flowtime_minimization(
    job_ops=job_ops,
    earliest_start=job_earliest_starts,
    schedule_start=0,
    solver_time_limit=max_time,
    msg=True,
    solver_relative_gap_limit=0.05,
    log_file=log_file_path
)
df_schedule = get_schedule_dframe(schedule)
df_schedule

In [ ]:
gantt.get_plot(df_schedule, perspective="Machine")
check.check_core_schedule_constraints(df_schedule)

check.is_start_correct(df_schedule, df_jobs_arrival_for_flowtime)
check.is_duration_correct(df_schedule)